# This code allow to run other ipynb codes on batch of data

## Processing

### Import Libraries

In [24]:
import papermill as pm
import ipywidgets as widgets

### Input Definition

In [25]:
total_samples = 5000

#### Season Selection

In [ ]:
date_s2_w = widgets.Dropdown(
    options=['Summer', 'Winter', 'Intermediate'],
    value='Summer',
    description='Sentinel-2 date:',
    disabled=False,
    layout={'width': 'max-content'},
    style = {'description_width': 'initial'}
)
date_s2_w

In [ ]:
date_s2 = date_s2_w.value

if (date_s2 == "Summer"):
    date_list=['2016-09-29',
                '2017-08-15',
                '2018-07-31',
                '2018-08-15',
                '2019-07-16',
                '2020-07-20',
                '2022-07-10',
                '2022-07-25',
                '2022-08-09',
                '2023-06-25',
                '2024-07-14',
                '2024-07-29']
    
elif (date_s2 == "Winter"):
    date_list=['2017-02-16',
                '2021-01-11',
                '2021-02-25',
                '2023-01-31',
                '2023-02-15',
                '2023-03-22',
                '2024-02-05']
else:
    date_list=['2015-11-24',
                '2017-04-22',
                '2017-10-14',
                '2017-11-18',
                '2018-04-22',
                '2018-10-19',
                '2020-04-11',
                '2020-11-22',
                '2021-10-13',
                '2023-11-17',
                '2024-05-10']
    
print("Dates Selected:", date_list)

### Processing

#### Samples Splitting

In [ ]:
# Step 1: Base samples per image
base = total_samples // len(date_list)
remainder = total_samples % len(date_list)

# Step 2: Create list of sample counts
samples_per_image_list = [base + 1 if i < remainder else base for i in range(len(date_list))]
print(f"Sample counts: {samples_per_image_list}")
print(f"Total: {sum(samples_per_image_list)}")

#### Samples Extraction

In [ ]:
for d, n_samples in zip(date_list, samples_per_image_list):
    print(f"Executing for {d} with {n_samples} samples")
    pm.execute_notebook(
        "\processing-checkpoints.ipynb",
        f"\output_processing\{d}_processing_{n_samples}_samples_{total_samples}.ipynb",
        parameters={
            "date_s2_w": d,
            "season": date_s2_w.value,
            "total_samples": n_samples,
            "combined" : total_samples
        },
    )

In [ ]:
import geopandas as gpd
import pandas as pd
import glob
import os

# Folder containing the per-image GPKG files
input_folder = "masks/"
output_file = "combined_sampled_points.gpkg"

# Pattern to match all sampled_points_*.gpkg files
gpkg_files = glob.glob(os.path.join(input_folder, f"{date_s2_w.value}_sampled_points_*.gpkg"))
print(len(gpkg_files))
# Load and combine all GeoPackages
gdf_list = []
for file in gpkg_files:
    print(f"Loading {file}")
    gdf = gpd.read_file(file)
    
    # Optional: extract the date from the filename and add it as a column
    date_str = os.path.basename(file).split("_")[-1].replace(".gpkg", "")
    #gdf["date"] = date_str
    
    gdf_list.append(gdf)

# Concatenate into a single GeoDataFrame
combined_gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True), crs=gdf_list[0].crs)

# Save combined file
combined_gdf.to_file(output_file, driver="GPKG")
print(f"\nCombined GeoPackage saved as: {output_file}")

# Model

## Model by season

### Import Libraries

In [2]:
import papermill as pm
import ipywidgets as widgets

### Input Definition

In [ ]:
total_samples = 5000

#### Season Selection

In [ ]:
season_s2_w = widgets.Dropdown(
    options=['Summer', 'Winter', 'Intermediate'],
    value='Summer',
    description='Season:',
    disabled=False,
    layout={'width': 'max-content'},
    style = {'description_width': 'initial'}
)
season_s2_w

In [ ]:
season_s2 = season_s2_w.value

if (season_s2 == "Summer"):
    date_s2=['2023-06-25']
elif (season_s2 == "Winter"):
    date_s2=['2023-03-22']
else:
    date_s2=['2023-11-17']
    
print("Dates Selected:", date_s2)

### Modelling

In [ ]:
pm.execute_notebook(
    "\model-training-season.ipynb",
    f"\output_model/{d}_model_{total_samples}_samples.ipynb",
    parameters={"date_s2": date_s2[0],
            "season": season_s2,
            "total_samples": total_samples
    },
)

## All seasonal models

### Import Libraries

In [ ]:
import papermill as pm

### Input Definition

In [ ]:
season_list = ["Summer", "Winter", "Intermediate"]
date_list = ['2023-06-25', '2023-03-22', '2023-11-17']

### Modelling

In [ ]:
for s in (season_list):
    pm.execute_notebook(
        "\model-training-season.ipynb",
        f"\output_model/{s}_model_{total_samples}_samples.ipynb",
        parameters={"date_s2": date_list[s],
                "season": season_list[s],
                "total_samples": total_samples
        },
    )